In [5]:
from typing import *

import numpy as np
from numpy.random import uniform

In [8]:
class HyperParameters:
    def __init__(self, 
                 num_params:int, 
                 limits: list[tuple[float,float]], 
                 parameters: np.ndarray[float]=None):
        if not isinstance(num_params, int):
            raise TypeError('num_params must be an integer.')
        if num_params <= 0:
            raise ValueError('num_params must be greater than 0.')
        if len(limits) != num_params:
            raise ValueError('list of limits should be of length "num_params". ')
        if parameters is not None:
            if len(parameters) != len(limits):
                raise ValueError('When explicitly provided, the number of parameters should be equal to "num_params". ')
        self.num_params = num_params
        self.limits = limits
        self.parameters = parameters
        
    def get_parameters(self):
        if self.parameters is None:
            raise RuntimeError(f' "parameters" attribute has not been set. ')
        return self.parameters
    
    def set_parameters(self, parameters: np.ndarray[float]):
        if len(parameters) != len(self.limits):
                raise ValueError('When explicitly provided, the number of parameters should be equal to "num_params". ')
        self.parameters = parameters
    
    def get_random_parameters(self, distribution: str = 'uniform'):
        if distribution == 'uniform':
            self.parameters = np.array([uniform(low=limit[0], high=limit[1], size=1) for limit in self.limits])
            return self.get_parameters()
        

class CovarianceMatrix:
    def __init__(self, kernel: Callable):
        self.kernel = kernel
        self.matrix = None
        self.samples = []
        
    def update_matrix(self, new_samples: list[np.ndarray[float]]):
        # Updating samples
        self.samples.extend(new_samples)
        
        # Filling matrix for first time
        if self.matrix is None:
            N = len(self.samples)
            matrix = np.empty((N, N), dtype=float)
            for i in range(N):
                for j in range(i,N):
                    matrix[i,j] = self.kernel(self.samples[i],self.samples[j])
                    if i != j:
                        matrix[j, i] = matrix[i, j]
            self.matrix = matrix
            
        # Updating existing matrix 
        else:
            N_new, N_old = len(new_samples), self.matrix.shape[0]
            N_total = N_new + N_old
            total_matrix = np.empty((N_total, N_total), dtype=float)
            
            # Updating total matrix w. old covariances
            total_matrix[:N_old,:N_old] = self.matrix
            
            # Calculating covariances between old and new samples
            for i in range(N_old):
                for j in range(N_new):
                    total_matrix[i, N_old + j] = self.kernel(self.samples[i], new_samples[j])
                    total_matrix[N_old + j, i] = total_matrix[i, N_old + j]
            
            # Calculating covariances between new samples
            for i in range(N_new):
                for j in range(i, N_new):
                    total_matrix[N_old + i, N_old + j] = self.kernel(new_samples[i], new_samples[j])
                    if i != j:
                        total_matrix[N_old + j, N_old + i] = total_matrix[N_old + i, N_old + j]
                       
    def get_matrix(self):
        if self.matrix is None:
            raise RuntimeError(f' "matrix" attribute has not yet been set. Method: ".update_matrix()" must be called first. ')
        return self.matrix
    

class GaussianProcess:
    def __init__(self):
        

    
def rbf_kernel(x1, x2, length_scale=1.0, sigma_f=1.0):
    """
    Compute the Radial Basis Function (RBF) kernel between two d-dimensional vectors.

    :param x1: First d-dimensional input vector.
    :param x2: Second d-dimensional input vector.
    :param length_scale: The length scale of the kernel.
    :param sigma_f: The signal variance (amplitude) of the kernel.
    :return: Scalar value representing the covariance between x1 and x2.
    """
    # Ensure the inputs are arrays (in case they are lists or tuples)
    x1 = np.asarray(x1)
    x2 = np.asarray(x2)
    
    # Compute the squared Euclidean distance between the vectors
    sqdist = np.sum((x1 - x2)**2)
    
    # Compute the RBF kernel
    return sigma_f**2 * np.exp(-0.5 / length_scale**2 * sqdist)  
        
        
    
def rosenbrock(x: float, y: float) -> float:
    return (1 - x) ** 2 + 100 * (y-x ** 2) ** 2

In [9]:
a, b = np.array([1,2]), np.array([3,2])
rbf_kernel(a,b)

0.1353352832366127

In [104]:
import numpy as np
A = np.array([['x_11', 'x_12', 'x_13', 'x_14'],
              ['x_21', 'x_22', 'x_23', 'x_24'],
              ['x_31', 'x_32', 'x_33', 'x_34'],
              ['x_41', 'x_42', 'x_43', 'x_44']], dtype=object)


In [105]:
B = np.array([['y_11', 'y_12'],
              ['y_21', 'y_22']],dtype=object)
B

array([['y_11', 'y_12'],
       ['y_21', 'y_22']], dtype=object)

In [106]:
n = B.shape[0]
A[2:2+n,2:2+n] = B

In [107]:
A

array([['x_11', 'x_12', 'x_13', 'x_14'],
       ['x_21', 'x_22', 'x_23', 'x_24'],
       ['x_31', 'x_32', 'y_11', 'y_12'],
       ['x_41', 'x_42', 'y_21', 'y_22']], dtype=object)

In [89]:
samples = ['d_1', 'd_2']
for i in range(A.shape[0]):
    for j in range(len(samples)):
        A[i,j+B.shape[0]] += samples[j]
        #if i != j:
        #    A[j+B.shape[0], i ] =  A[i,j+B.shape[0]]
    for j in range(len(samples)):
        A[i,j+B.shape[0]] += samples[j]

In [90]:
A

array([['y_11', 'y_12', 'x_13d_1', 'x_14d_2'],
       ['y_21', 'y_22', 'x_23d_1', 'x_24d_2'],
       ['x_31', 'x_32', 'x_33d_1', 'x_34d_2'],
       ['x_41', 'x_42', 'x_43d_1', 'x_44d_2']], dtype=object)